In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error, r2_score,mean_absolute_error
from sklearn.preprocessing import StandardScaler
import joblib

In [3]:
df = pd.read_csv('../../data/training_data.csv', sep=',')
df.drop(columns=['Rating Average'],inplace=True)
df

,Min Players,Max Players,Play Time,Min Age,Users Rated,Complexity Average,Owned Users,Abstract Games,Children's Games,Customizable Games,...,Wargames,Social Interaction,Strategy and Planning,Conflict Resolution,Exploration,Randomness and Luck,Resource Management,Puzzle Solving,Miscellaneous / Other,Collaboration
0,1,4,120.000000,14,42055,3.8604,68323.0,0,0,0,...,0,0,1,0,1,0,1,1,0,1
1,2,4,60.000000,13,41643,2.8405,65294.0,0,0,0,...,0,0,1,0,1,0,1,1,0,1
2,2,4,120.000000,14,19217,3.9129,28785.0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
3,1,5,120.000000,12,64864,3.2406,87099.0,0,0,0,...,0,0,1,1,0,0,1,1,0,0
4,3,6,431.858995,14,13468,4.2219,16831.0,0,0,0,...,0,1,1,1,0,1,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20315,2,2,30.000000,4,1340,1.0000,427.0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
20316,2,99,60.000000,5,2154,1.0455,1533.0,0,0,0,...,0,0,0,0,0,1,0,1,0,0
20317,2,4,30.000000,3,4006,1.0779,5788.0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
20318,2,6,30.000000,3,3783,1.0201,4400.0,0,1,0,...,0,0,0,0,0,1,0,0,0,0


In [4]:
X = df.drop(columns=['Complexity Average'])
y = df['Complexity Average']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestRegressor(random_state=42))
])

In [5]:
param_grid = {
    'rf__n_estimators': [50, 100, 150, 200, 250, 300],  
    'rf__max_depth': [5, 10, 20, 30, 40, 50], 
    'rf__min_samples_split': [2, 5, 10], 
    'rf__min_samples_leaf': [1, 2, 5, 10], 
    'rf__bootstrap': [True, False]  
}

grid_search = GridSearchCV(
    estimator=pipeline,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    cv=5,
    n_jobs=-1,
    verbose=2
)

# Fit grid search
grid_search.fit(X_train, y_train)

# Best parameters
best_params = grid_search.best_params_
print(f"Best hyperparameters: {best_params}")

Fitting 5 folds for each of 864 candidates, totalling 4320 fits
Best hyperparameters: {'rf__bootstrap': True, 'rf__max_depth': 20, 'rf__min_samples_leaf': 5, 'rf__min_samples_split': 2, 'rf__n_estimators': 300}


In [6]:
y_pred = grid_search.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"Mean Absolute Error: {mae}")
print(f"R² Score: {r2}")

Mean Squared Error: 0.2332649392974472
Mean Absolute Error: 0.3361228832428197
R² Score: 0.6706322117493393


In [7]:
joblib.dump(grid_search.best_estimator_, '../../models/complexity_forest.pkl')

['../../models/complexity_forest.pkl']